In [9]:
import os
import numpy as np
import tifffile
from roifile import ImagejRoi
from skimage import draw
from pathlib import Path

def process_tiff_with_rois(tiff_path, output_folder):
    # Read the TIFF file
    tiff_image = tifffile.imread(tiff_path)

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    roi_files = []
    for file in os.listdir(Path(tiff_path).parent):
        if file.endswith(".roi"):
            file = os.path.join(Path(tiff_path).parent, file)
            roi_files.append(file)
    roi_dir = os.path.join(Path(tiff_path).parent, "cells")
    if Path(roi_dir).exists() and os.path.isdir(roi_dir):
        for item in os.listdir(roi_dir):
            if not item.endswith(".roi"):
                continue
            file = os.path.join(roi_dir, item)
            roi_files.append(file)
    for (index, roi_file) in enumerate(roi_files): 
        print(roi_file, "3554.roi" not in roi_file)
        if "3554.roi" not in roi_file:
            continue
        # Read the ROI file
        roi = ImagejRoi.fromfile(roi_file)

        # Create a mask from the ROI
        mask = np.zeros(tiff_image.shape[:2], dtype=bool)
        
        if roi.roitype == 7:  # Freehand ROI
            # Get the coordinates of the freehand ROI
            coords = roi.coordinates()
            x_coords, y_coords = coords[:, 0], coords[:, 1]

            min_x, max_x = int(np.min(x_coords)), int(np.max(x_coords))
            min_y, max_y = int(np.min(y_coords)), int(np.max(y_coords))
            
            # Create a mask from the ROI
            mask = np.zeros((max_y - min_y + 1, max_x - min_x + 1), dtype=bool)
            rr, cc = draw.polygon(y_coords - min_y, x_coords - min_x, mask.shape)
            mask[rr, cc] = True

            # Crop the original image to the ROI bounding box
            cropped_image = tiff_image[min_y:max_y+1, min_x:max_x+1]
            if cropped_image.ndim == 2:  # Grayscale image
                masked_image = cropped_image * mask

            # Then copy over the c13c12 ratio map cropped
            c13c12_ratio = tifffile.imread(os.path.join(Path(tiff_path).parent, "13C12C_ratio_final.tiff"))
            cropped_ratio = c13c12_ratio[min_y:max_y+1, min_x:max_x+1].astype(np.uint16)
            print(roi_file, os.path.join(Path(tiff_path).parent, "13C12C_ratio_final.tiff"), min_y, max_y, min_x, max_x)
            tifffile.imwrite(os.path.join(output_folder, "output_c13c12", f"{Path(roi_file).stem}222.tiff"), cropped_ratio)
        else:
            print(f"Unsupported ROI type ({roi.roitype}) for {roi_file}. Skipping.")
            continue

        # Save the masked image
        output_path = os.path.join(output_folder, "output_cells", f"{Path(roi_file).stem}.tiff")
        #tifffile.imwrite(output_path, masked_image)

        print(f"Processed {roi_file}")

    print("All ROIs processed successfully.")


In [10]:
import os
import shutil

base_dir = "/Users/chris/Documents/lab/CD2_mouse_data"
output_dir = os.path.join(base_dir, "outputs")

for roi in range(12, 16):
    folder = os.path.join(base_dir, f"roi{roi}")
    # Usage
    tiff_path = os.path.join(folder, "sem.tiff")
    process_tiff_with_rois(tiff_path, output_dir)
    
    


/Users/chris/Documents/lab/CD2_mouse_data/roi12/3393-3142.roi True
All ROIs processed successfully.
/Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/4579-5132.roi True
/Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/3251-2860.roi True
/Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/6562-3554.roi False
/Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/6562-3554.roi /Users/chris/Documents/lab/CD2_mouse_data/roi13/13C12C_ratio_final.tiff 5227 7897 1433 5675
Processed /Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/6562-3554.roi
/Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/5339-2448.roi True
/Users/chris/Documents/lab/CD2_mouse_data/roi13/cells/2496-5331.roi True
All ROIs processed successfully.
/Users/chris/Documents/lab/CD2_mouse_data/roi14/cells/5307-2916.roi True
/Users/chris/Documents/lab/CD2_mouse_data/roi14/cells/4617-5145.roi True
All ROIs processed successfully.
/Users/chris/Documents/lab/CD2_mouse_data/roi15/1968-3870.roi True
All ROIs processe